# NAO Prediction

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import pymongo
from pprint import pprint
from datetime import datetime, timedelta, date
import pandas as pd
from sklearn.decomposition import PCA
import sklearn.linear_model as skl_lm
import gdal as gdl
import matplotlib.mlab as ml
import cartopy.crs as ccrs
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True) # for live plot
pd.set_option('display.notebook_repr_html', False)
%matplotlib inline
plt.style.use('seaborn-white')

In [2]:
mongo_host_local = 'mongodb://localhost:27017/'
mg = pymongo.MongoClient(mongo_host_local)

In [3]:
db = mg.ECMWF
db.collection_names()

['system.indexes',
 'ERAINT_grid',
 'ERAINT_lores_grid',
 'ERAINT_lores_monthly_anom',
 'ERAINT_monthly',
 'ERAINT_lores_monthly']

In [4]:
ERA_vers = 'lores'
if (ERA_vers == 'hires'):
    col_dat = 'ERAINT_monthly'
    col_anom = 'ERAINT_monthly_anom'
    col_grid = 'ERAINT_grid'
    resolution = 0.25
elif (ERA_vers == 'lores'):
    col_dat = 'ERAINT_lores_monthly'
    col_anom = 'ERAINT_lores_monthly_anom'
    col_grid = 'ERAINT_lores_grid'
    resolution = 2.5

# Construct NAO index

In [5]:
# Query grid cells for NAO calculation
con_grid = db[col_grid]
poly1 = [list(reversed([ [-50,25], [-50,55], [10,55],[ 10,25], [-50,25]]))]
poly2 = [list(reversed([ [-40, 55], [-40, 85], [20, 85], [20, 55], [-40, 55]]))]
def getGridIds(this_polygon):
    geo_qry = {"loc": 
               {"$geoWithin": {
                   "$geometry": {
                       "type": "Polygon",
                       "coordinates": this_polygon
                   }
               }}}

    res = con_grid.find(filter = geo_qry, projection = {"_id":0, "id_grid": 1, "loc": 1})
    grid_df = pd.DataFrame(list(res))
    return grid_df
grid_df1 = getGridIds(poly1)
grid_ids1 = grid_df1.id_grid.values
grid_df2 = getGridIds(poly2)
grid_ids2 = grid_df2.id_grid.values

In [6]:
# Get yearly DJF averages over the two NAO nodal locations

con_anom = db[col_anom]

def setWinterYear(date): # December belong to next year's winter
    mon=date.month
    yr=date.year
    if mon >= 9:
        res = yr+1
    else:
        res = yr
    return res

def getMSL(this_grid_ids):
    this_msl = con_anom.aggregate(pipeline=[
        {"$match": {"id_grid": {"$in": this_grid_ids.tolist()}}},
        {"$group": {"_id": "$date", "mean": {"$avg": "$msl"} }},
        {"$project": {"date": "$_id", 
                      "_id": 0, 
                      "msl": "$mean"}}])
    this_msl_df = pd.DataFrame(list(this_msl))
    this_msl_df = this_msl_df.assign(
            month=list(map(lambda x: x.month, this_msl_df.date)),
            wyear=list(map(lambda x: setWinterYear(x), this_msl_df.date)) ).pipe(
    lambda df: df.query("month in [12, 1, 2]") ).pipe(
    lambda df: df.groupby("wyear").mean().reset_index())
    return this_msl_df
    
msl1_df = getMSL(this_grid_ids = grid_ids1).rename(columns={'msl': 'msl1'})
msl2_df = getMSL(this_grid_ids = grid_ids2).rename(columns={'msl': 'msl2'})
msl_df = pd.merge(msl1_df, msl2_df)
msl_df = msl_df.assign(NAO = msl_df.msl2-msl_df.msl1).sort_values('wyear', ascending=True).reset_index(drop=True)
# Get rid of the first year (1979) and the last (2017) because the winter month are not complete
msl_df = msl_df.query("(wyear > 1979) & (wyear <2017)")
msl_df.head()

   wyear        msl1  month        msl2         NAO
1   1980  152.977452    5.0  -98.370800 -251.348252
2   1981 -444.747185    5.0  117.529424  562.276609
3   1982  336.279706    5.0 -377.140170 -713.419876
4   1983 -342.535287    5.0  378.676276  721.211563
5   1984 -220.904942    5.0  330.200922  551.105864

In [7]:
# Plot ts
data = [go.Scatter(x=msl_df['wyear'], y=msl_df['NAO'] )]
py.iplot(data, filename='pandas-time-series')

In [8]:
# Query anomalies for a variable for each input grid cells
def queryAnom(this_variable, this_grid_df):
    # Query data anomalies
    grid_ids = this_grid_df.id_grid.values
    res = con_anom.aggregate(pipeline=[ 
    {"$project": {"id_grid": 1, "date": 1, this_variable: 1, "month": {"$month": "$date"}}},
    {"$match": {"month": {"$in": [9, 10, 11, 12, 1, 2]},
                "id_grid": {"$in": grid_ids.tolist()} }},
    {"$project": {"_id": 0, "id_grid": 1, "date": 1, this_variable: 1}} ])    
    anom_df = pd.DataFrame(list(res))
    return anom_df

In [9]:
# Region to retrieve Niño 3.4 index for SST
# Niño 3.4 region: stretches from the 120th to 170th meridians west longitude astride 
# the equator five degrees of latitude on either side (Wikipedia)
poly_Nino = [list(reversed([ [-170,-5], [-170,5],[-120,5], [-120,-5], [-170,-5]]))]
grid_df_Nino = getGridIds(poly_Nino)
grid_ids_Nino = grid_df_Nino.id_grid.values
anom_sst_df = queryAnom(this_variable='sst', this_grid_df=grid_df_Nino)
nino_df0 = anom_sst_df[['date', 'sst']].groupby('date').mean().reset_index().rename(columns={'sst':'Nino'})
nino_df0.head()

        date      Nino
0 1979-01-01  0.149061
1 1979-02-01  0.114270
2 1979-09-01 -0.320141
3 1979-10-01 -0.224291
4 1979-11-01 -0.295341

# Get PCA scores

In [10]:
# Generic function to query grid ids above a given latitude
def genCircle(start_lon, stop_lon, lat, decreasing): 
    res = map(lambda x:[int(x), lat],
              sorted(np.arange(start=start_lon, stop=stop_lon+1), reverse=decreasing))
    return list(res)

def queryGrids(aboveLat):
    this_box = {'lonmin': -180, 'lonmax': 180, 'latmin': aboveLat, 'latmax': 90}
    circle_north_pos = genCircle(start_lon = this_box['lonmin'], stop_lon = this_box['lonmax'], 
                                  lat = this_box['latmax'], decreasing = False)
    circle_south_neg = genCircle(start_lon = this_box['lonmin'], stop_lon = this_box['lonmax'], 
                                lat = this_box['latmin'],  decreasing = True)
    slp_poly = [[this_box['lonmin'], this_box['latmin']]]
    slp_poly.extend(circle_north_pos)
    slp_poly.extend(circle_south_neg)
    this_polygon = slp_poly
    
    if aboveLat > 0:
        geo_qry = {"loc": 
               {"$geoWithin": {
                   "$geometry": {
                       "type": "Polygon",
                       "coordinates": [this_polygon]
               }}}}
    else: # case of a big polygon larger than one hemisphere
        geo_qry = {"loc": 
               {"$geoWithin": {
                   "$geometry": {
                       "type": "Polygon",
                       "coordinates": [list(reversed(this_polygon))], # the orientation matters
                       "crs": {
                           "type": "name", 
                           "properties": { "name": "urn:x-mongodb:crs:strictwinding:EPSG:4326" }
                       }
                   }
               }}}
        
    res = con_grid.find(filter = geo_qry, projection = {"_id":0, "id_grid": 1, "loc": 1})
    grid_df = pd.DataFrame(list(res))
    return grid_df

grid_df_20N = queryGrids(aboveLat=20)
grid_df_20S = queryGrids(aboveLat=-20)

In [11]:
# 3rd region for SST in Northern Atlantic, as in Promet (2008).
poly_NAtlantic = [list(reversed(
    [ [-100,0], [-100,45],[-100,89], [-40, 89],[20,89],[20,45],[20,0], [-40,0], [-100,0]]))]
grid_df_NAtlantic = getGridIds(poly_NAtlantic)
grid_ids_NAtlantic = grid_df_NAtlantic.id_grid.values

In [12]:
def queryScores(this_variable, this_grid_df):
    # Query data anomalies
    anom_df = queryAnom(this_variable, this_grid_df)
    # Get Principal Component Scores
    X_df = anom_df.pivot(index='date', columns='id_grid', values=this_variable)
    pca = PCA(n_components=3)
    df_scores = pd.DataFrame(pca.fit_transform(X_df), 
                             columns=['PC1_%s' % (this_variable), 
                                      'PC2_%s' % (this_variable), 
                                      'PC3_%s' % (this_variable)],
                             index=X_df.index)
    return df_scores

scores_z70 = queryScores(this_variable='z70', this_grid_df=grid_df_20N)
scores_ci = queryScores(this_variable='ci', this_grid_df=grid_df_20N)
scores_sst = queryScores(this_variable='sst', this_grid_df=grid_df_20S)
scores_sst_NAtl = queryScores(this_variable='sst', this_grid_df=grid_df_NAtlantic)

In [13]:
scores_sst_NAtl = scores_sst_NAtl.rename(columns={"PC1_sst": "PC1_sstna",
                                                   "PC2_sst": "PC2_sstna",
                                                   "PC3_sst": "PC3_sstna"})

### Comparison with correlation coefficients

In [14]:
P03_df = scores_ci.assign(month=list(map(lambda x: x.month, scores_ci.index))).query('month == 10')
P03_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 38 entries, 1979-10-01 to 2016-10-01
Data columns (total 4 columns):
PC1_ci    38 non-null float64
PC2_ci    38 non-null float64
PC3_ci    38 non-null float64
month     38 non-null int64
dtypes: float64(3), int64(1)
memory usage: 1.5 KB


In [15]:
np.corrcoef(P03_df.PC1_ci[:-1], msl_df.NAO)

array([[ 1.        , -0.49539095],
       [-0.49539095,  1.        ]])

In [16]:
P04_df = scores_z70.assign(month=list(map(lambda x: x.month, scores_z70.index))).query('month == 10')
np.corrcoef(P04_df.PC2_z70[:-1], msl_df.NAO)

array([[ 1.        , -0.43572617],
       [-0.43572617,  1.        ]])

In [17]:
P17_df = scores_sst.assign(month=list(map(lambda x: x.month, scores_sst.index))).query('month == 9')
np.corrcoef(P17_df.PC3_sst[:-1], msl_df.NAO)

array([[ 1.        ,  0.04698433],
       [ 0.04698433,  1.        ]])

### Group all predictors in one DataFrame

In [18]:
def assignWyear(df):
    res_df = df.assign(
    year=list(map(lambda x: x.year, df.date)),
    wyear=list(map(lambda x: setWinterYear(x), df.date)), 
    month=list(map(lambda x: x.month, df.date)))
    return res_df

In [19]:
scores_df = pd.merge(left=scores_z70, right=scores_ci, left_index=True, right_index=True).\
pipe(lambda df: pd.merge(df, scores_sst, left_index=True, right_index=True)).\
pipe(lambda df: pd.merge(df, scores_sst_NAtl, left_index=True, right_index=True))
scores_df.reset_index(level=0, inplace=True)
scores_df0 = assignWyear(df=scores_df)
nino_df = assignWyear(df=nino_df0)
scores_df = pd.merge(scores_df0, nino_df)
scores_df.head()

        date       PC1_z70       PC2_z70       PC3_z70    PC1_ci    PC2_ci  \
0 1979-01-01  47305.944044 -21384.693316 -20280.483751 -0.069347  0.698022   
1 1979-02-01  27377.090406  -5627.629663 -24174.825459 -0.176903  0.691111   
2 1979-09-01  18023.465139   2794.736512  -2092.985110  2.963272  1.071392   
3 1979-10-01  14187.255820  -8848.225741  -3218.127655  2.152409  0.879826   
4 1979-11-01  27520.923158 -14059.658557  -5650.416198  2.057799 -0.180304   

     PC3_ci   PC1_sst   PC2_sst   PC3_sst  PC1_sstna  PC2_sstna  PC3_sstna  \
0  1.563147 -3.052552  3.453180 -5.779823   6.061390  -9.033219   4.809590   
1  1.377238 -6.587935  8.314599 -7.697487  10.096280  -8.582903   5.170891   
2  1.375140  5.315733  5.144099 -0.101496   6.234057  -2.363909   5.491280   
3 -0.169622  6.340731  3.405892  3.130245   1.479563  -6.115383   5.599424   
4 -0.983038  3.580924  3.571938  7.027206   1.090428  -3.748723  10.074660   

   year  wyear  month      Nino  
0  1979   1979      1  0.149

In [20]:
# Create the Predictor DataFrame
def renCol(x, mon):
    if ('PC' in x or 'Nino' in x):
        z = '%s_%s' % (x, mon)
    else:
        z = x
    return z

def createMondf(this_mon, scores_df):
    mon_df = scores_df.query('month == @this_mon')
    mon_df.columns = list(map(lambda x: renCol(x, mon=this_mon), list(mon_df)))
    mon_df = mon_df.drop(['date','year','month'], axis=1)
    return mon_df

sep_df = createMondf(this_mon=9, scores_df=scores_df)
oct_df = createMondf(this_mon=10, scores_df=scores_df)
X_df = pd.merge(sep_df, oct_df)

In [21]:
# Create Regression DataFrame
NAO_df = msl_df.drop(columns=['msl1', 'msl2', 'month'])
dat_df = pd.merge(NAO_df, X_df)
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 36
Data columns (total 28 columns):
wyear           37 non-null int64
NAO             37 non-null float64
PC1_z70_9       37 non-null float64
PC2_z70_9       37 non-null float64
PC3_z70_9       37 non-null float64
PC1_ci_9        37 non-null float64
PC2_ci_9        37 non-null float64
PC3_ci_9        37 non-null float64
PC1_sst_9       37 non-null float64
PC2_sst_9       37 non-null float64
PC3_sst_9       37 non-null float64
PC1_sstna_9     37 non-null float64
PC2_sstna_9     37 non-null float64
PC3_sstna_9     37 non-null float64
Nino_9          37 non-null float64
PC1_z70_10      37 non-null float64
PC2_z70_10      37 non-null float64
PC3_z70_10      37 non-null float64
PC1_ci_10       37 non-null float64
PC2_ci_10       37 non-null float64
PC3_ci_10       37 non-null float64
PC1_sst_10      37 non-null float64
PC2_sst_10      37 non-null float64
PC3_sst_10      37 non-null float64
PC1_sstna_10    37 non-null float64

In [22]:
dat_df.head()

   wyear         NAO     PC1_z70_9    PC2_z70_9     PC3_z70_9  PC1_ci_9  \
0   1980 -251.348252  18023.465139  2794.736512  -2092.985110  2.963272   
1   1981  562.276609  -5422.104718  9689.567172   5922.630486  0.101697   
2   1982 -713.419876  19744.941093 -8460.604205   1635.209243 -0.538567   
3   1983  721.211563 -14530.470578  8700.189280  -4888.437956 -0.840921   
4   1984  551.105864  -5774.518779  2361.237107  13897.025591  0.980549   

   PC2_ci_9  PC3_ci_9  PC1_sst_9  PC2_sst_9    ...     PC1_ci_10  PC2_ci_10  \
0  1.071392  1.375140   5.315733   5.144099    ...      2.152409   0.879826   
1  0.728209  0.115510  -3.156064   0.330398    ...      0.352610   0.479747   
2  1.527346 -1.105449  -4.273700   2.978050    ...      0.624417   0.928135   
3  1.564581 -0.486861  19.597934   7.319275    ...     -1.098517   1.081049   
4  0.753583  0.899558   2.394603  -2.069533    ...      1.588658  -0.114038   

   PC3_ci_10  PC1_sst_10  PC2_sst_10  PC3_sst_10  PC1_sstna_10  PC2_sstna_

# Regression

In [23]:
regr = skl_lm.LinearRegression()
X = dat_df[['PC1_ci_10', 
            'PC2_z70_10',
            'PC3_sst_9']].as_matrix()
y = dat_df.NAO
regr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [24]:
regr.score(X, y)

0.50037140298348137

In [25]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

/home/dmasson/anaconda3/envs/winter_predictor/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [26]:
est = smf.ols('NAO ~ PC1_ci_10 + PC2_z70_10 + PC3_sst_9', dat_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    NAO   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.455
Method:                 Least Squares   F-statistic:                     11.02
Date:                Sun, 11 Mar 2018   Prob (F-statistic):           3.63e-05
Time:                        13:29:17   Log-Likelihood:                -279.32
No. Observations:                  37   AIC:                             566.6
Df Residuals:                      33   BIC:                             573.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.6814     80.217      0.046      0.964    -159.521     166.884
PC1_ci_10   -289.4934     64.195     -4.510      0.000    -420.099    -158.887
PC2_z70_10    -0.0208      0.007     -2.909      0.006      -0.035      -0.006
PC3_sst_9     31.2633     11.542      2.709      0.011       7.781      54.746
==============================================================================
Omnibus:                        2.213   Durbin-Watson:                   2.305
Prob(Omnibus):                  0.331   Jarque-Bera (JB):                1.717
Skew:                          -0.359   Prob(JB):                        0.424
Kurtosis:                       2.226   Cond. No.                     1.14e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.14e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
# Complete Overfitted Model:
#est = smf.ols('NAO ~ PC1_z70_9 + PC2_z70_9 + PC3_z70_9 + PC1_ci_9 + PC2_ci_9 + PC3_ci_9 + PC1_sst_9 + PC2_sst_9 + PC3_sst_9 + PC1_z70_10 + PC2_z70_10 + PC3_z70_10 + PC1_ci_10 + PC2_ci_10 + PC3_ci_10 + PC1_sst_10 + PC2_sst_10 + PC3_sst_10', dat_df).fit()
#est.summary()

## Regularization / Lasso Model Selection

In [28]:
# Predictors:
# 'NAO ~ PC1_ci_10 + PC2_z70_10 + PC3_sst_9' # Wang
predNames = np.array(['PC1_z70_9',
 'PC2_z70_9',
 'PC3_z70_9', # selected by Lasso
 'PC1_ci_9',
 'PC2_ci_9',
 'PC3_ci_9',
 'PC1_sst_9',
 'PC2_sst_9',
 'PC3_sst_9', # selected by Lasso
 'PC1_z70_10', # selected by Lasso 
 'PC2_z70_10', # selected by Lasso & as in Wang et al. 2010
 'PC3_z70_10',
 'PC1_ci_10', # selected by Lasso & as in Wang et al. 2010
 'PC2_ci_10',
 'PC3_ci_10',
 'PC1_sst_10',
 'PC2_sst_10',
 'PC3_sst_10',
 'PC1_sstna_10','PC2_sstna_10','PC3_sstna_10',
                      'Nino_9', 'Nino_10'])
X = dat_df[predNames].as_matrix()
# Target Variables:
y = dat_df.NAO

In [29]:
from sklearn.preprocessing import StandardScaler
# Before applying the Lasso, it is necessary to standardize the predictor
scaler = StandardScaler()
scaler.fit(X)
X_stan = scaler.transform(X)

In [30]:
# Lasso Regression with fixed penalty term lambda=150:
# We see that all predictors but three have been shrunk to null:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=150)
clf.fit(X_stan, y)
print(clf.coef_)

[  -0.            0.           79.24917277   -0.           -0.            0.
   -0.           -0.            0.           -0.          -92.0394126     0.
 -178.33922105   -0.           -0.           -0.           -0.            0.
   -0.            0.           -0.            0.            0.        ]


In [38]:
# In order to find the optimal penalty parameter alpha,
# use Cross-validated Lasso
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
#modlcv = LassoLarsIC(criterion='aic')
modlcv = LassoCV(cv=3, n_alphas=10000,max_iter=10000)
modlcv.fit(X_stan, y)
alpha = modlcv.alpha_
alpha # Optimal penalty

81.855260729962339

In [39]:
# Non-zero predictors:
modlcv.coef_

array([   0.        ,    0.        ,  119.82545763,    0.        ,
         -0.        ,    0.        ,   -0.        ,   -0.        ,
         23.50774767,   -2.24653301, -155.2916353 ,    0.        ,
       -267.45638045,    0.        ,  -10.76823728,   -0.        ,
          0.        ,    0.        ,  -64.43287177,    0.        ,
         -0.        ,    0.        ,    0.        ])

In [40]:
# Model R^2 :
modlcv.score(X_stan, y)

0.51554966157004301

In [41]:
# Name Of the non-null coefficients:

# 'NAO ~ PC1_ci_10 + PC2_z70_10 + PC3_sst_9' # Wang
ind = np.array(list(map(lambda x: int(x)!=0, modlcv.coef_)))
importance_df = pd.DataFrame({'pred': predNames[ind], 
                              'coef': modlcv.coef_[ind]})
importance_df = importance_df.assign(absCoef=np.absolute(importance_df.coef))
# According to the Lasso, the 3 strongest predictors are:
# PC3_z70_9, PC2_z70_10, PC1_ci_10
importance_df.sort_values('absCoef', ascending=False)

         coef          pred     absCoef
4 -267.456380     PC1_ci_10  267.456380
3 -155.291635    PC2_z70_10  155.291635
0  119.825458     PC3_z70_9  119.825458
6  -64.432872  PC1_sstna_10   64.432872
1   23.507748     PC3_sst_9   23.507748
5  -10.768237     PC3_ci_10   10.768237
2   -2.246533    PC1_z70_10    2.246533

We see that the Nino does not seem to play any role.

In [35]:
# Let's repeat the linear regression using the first 4 best predictors suggested by the Lasso:
#est = smf.ols('NAO ~ PC1_ci_10 + PC2_z70_10 + PC3_sst_9', dat_df).fit() # Wang
est = smf.ols('NAO ~ PC3_z70_9 + PC2_z70_10 + PC1_ci_10 + PC1_sstna_10', dat_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    NAO   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     10.64
Date:                Sun, 11 Mar 2018   Prob (F-statistic):           1.34e-05
Time:                        13:30:08   Log-Likelihood:                -276.51
No. Observations:                  37   AIC:                             563.0
Df Residuals:                      32   BIC:                             571.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       13.0319     75.625      0.172      0.864    -141.012     167.076
PC3_z70_9        0.0246      0.013      1.848      0.074      -0.003       0.052
PC2_z70_10      -0.0221      0.008     -2.895      0.007      -0.038      -0.007
PC1_ci_10     -248.3216     53.464     -4.645      0.000    -357.224    -139.420
PC1_sstna_10   -29.5680     14.347     -2.061      0.048     -58.791      -0.345
==============================================================================
Omnibus:                        5.568   Durbin-Watson:                   2.292
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                2.762
Skew:                          -0.411   Prob(JB):                        0.251
Kurtosis:                       1.943   Cond. No.                     1.16e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
np.sqrt(0.571)

In [ ]:
# Next step: Do Lasso Regression based on all possible ERA-int variables
